In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Summary

Start compiling humor dataset(s) for RL.

In [78]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import pandas as pd
from datasets import Dataset, load_dataset

from aeon.logging import logger

In [15]:
base_url = "https://scrapsfromtheloft.com/stand-up-comedy-scripts/"
url_prefix = 'https://scrapsfromtheloft.com/comedy/'

In [4]:
r = requests.get(base_url)

In [5]:
r

<Response [200]>

In [6]:
soup = BeautifulSoup(r.content, 'lxml')

In [47]:
transcript_urls = list(set(
    [link['href'] for link in soup.find_all('a')
    if link['href'].startswith(url_prefix)
    and link['href'] != url_prefix]
))

In [48]:
len(transcript_urls)

499

In [71]:
sess = requests.Session()

In [74]:
def _fetch_standup_transcript(url: str, sess: requests.Session) -> str:
    try:
        r = sess.get(url, timeout=5)
        r.raise_for_status()
    except Exception as e:
        logger.error(f'Failed to fetch url: {url} (error: {e})')
        return ''
    
    soup = BeautifulSoup(r.content, 'lxml')
    return '\n\n'.join(
        para.text 
        for para in soup.select("div.elementor-widget-container > p")
    )

fetch_standup_transcript = partial(_fetch_standup_transcript, sess=sess)

In [75]:
with ThreadPoolExecutor(max_workers=20) as exc:
    transcripts = list(tqdm(
        exc.map(fetch_standup_transcript, transcript_urls),
        total=len(transcript_urls)
    ))

  0%|                                                                                                  | 0/499 [00:00<?, ?it/s]ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/stewart-lee-90s-comedian-2006-full-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/stewart-lee-90s-comedian-2006-full-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/ellen-degeneres-the-beginning-2000-full-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/ellen-degeneres-the-beginning-2000-full-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/neal-brennan-women-and-black-dudes-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/neal-brennan-women-and-black-dudes-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/c

ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/fortune-feimster-good-fortune-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/fortune-feimster-good-fortune-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/jim-jefferies-alcoholocaust-2010-full-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/jim-jefferies-alcoholocaust-2010-full-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/dave-chappelle-acceptance-speech-2019-mark-twain-prize/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/dave-chappelle-acceptance-speech-2019-mark-twain-prize/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/eddie-izzard-dress-kill-1999-full-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfr

ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/dave-chappelle-846-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/dave-chappelle-846-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/billy-connolly-high-horse-tour-live-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/billy-connolly-high-horse-tour-live-transcript/)
 11%|█████████▋                                                                               | 54/499 [00:03<00:32, 13.72it/s]ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/mo-amer-mohammed-in-texas-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/mo-amer-mohammed-in-texas-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/chelsea-handler-the-feeling-transcript/ (error: 503 

ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/ralphie-may-filthy-animal-tour-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/ralphie-may-filthy-animal-tour-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/john-mulaney-snl-monologue-2018-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/john-mulaney-snl-monologue-2018-transcript/)
 17%|███████████████▌                                                                         | 87/499 [00:04<00:13, 31.21it/s]ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/george-carlin-dumb-americans-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/george-carlin-dumb-americans-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/gary-gulman-born-on-3rd-base-t

ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/sammy-obeid-how-to-save-gaza-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/sammy-obeid-how-to-save-gaza-transcript/)
 24%|████████████████████▊                                                                   | 118/499 [00:04<00:08, 43.59it/s]ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/hannah-berner-we-ride-at-dawn-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/hannah-berner-we-ride-at-dawn-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/eddie-izzard-glorious-1997-full-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/eddie-izzard-glorious-1997-full-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/sebastian-maniscalco-whats-wrong

...

ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/george-carlin-you-are-all-diseased-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/george-carlin-you-are-all-diseased-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/michelle-wolf-the-well-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/michelle-wolf-the-well-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/kevin-bridges-whole-different-story-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/kevin-bridges-whole-different-story-transcript/)
 72%|███████████████████████████████████████████████████████████████▎                        | 359/499 [00:11<00:04, 30.22it/s]ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/patton-oswalt-we-all-screa

ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/dave-chappelle-hbo-half-hour-1998-traduzione-italiana/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/dave-chappelle-hbo-half-hour-1998-traduzione-italiana/)
 78%|████████████████████████████████████████████████████████████████████▌                   | 389/499 [00:12<00:02, 38.88it/s]ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/w-kamau-bell-private-school-negro-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/w-kamau-bell-private-school-negro-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/amy-schumer-growing-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/amy-schumer-growing-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/eric-andre-legalize-

ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/mike-epps-only-one-mike-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/mike-epps-only-one-mike-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/enissa-amani-ehrenwort-2018-full-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/enissa-amani-ehrenwort-2018-full-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/daniel-sloss-x-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/daniel-sloss-x-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/david-spade-dandelion-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/david-spade-dandelion-transcript/)
ERROR:aeon.logging:Failed to fetch u

ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/kevin-james-irregardless-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/kevin-james-irregardless-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/nate-bargatze-nashville-christmas-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/nate-bargatze-nashville-christmas-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/t-j-miller-no-real-reason-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/t-j-miller-no-real-reason-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/louis-c-k-sorry-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/louis-c-k-sorry-transcript/)
ERROR:aeon.logging:Fai

ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/theo-von-no-offense-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/theo-von-no-offense-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/jimmy-carr-funny-business-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/jimmy-carr-funny-business-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/sam-morril-youve-changed-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/sam-morril-youve-changed-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scrapsfromtheloft.com/comedy/mae-martin-sap-transcript/ (error: 503 Server Error: Service Unavailable for url: https://scrapsfromtheloft.com/comedy/mae-martin-sap-transcript/)
ERROR:aeon.logging:Failed to fetch url: https://scra

In [76]:
sum(bool(row) for row in transcripts)

7

In [69]:
pd.DataFrame({"url": transcript_urls, "transcript": transcripts}).iloc[10:50]

,url,transcript
10,https://scrapsfromtheloft.com/comedy/kevin-bri...,♪ I’m at the crossroads ♪\n♪ Getting drowned i...
11,https://scrapsfromtheloft.com/comedy/ali-wong-...,[“Get me Bodied (Extended Mix)” playing]\n\n[a...
12,https://scrapsfromtheloft.com/comedy/stewart-l...,"Recorded on 10 March 2006 at Chapter Arts, Can..."
13,https://scrapsfromtheloft.com/comedy/adam-devi...,[rock music playing]\n\n[indistinct chatter]\n...
14,https://scrapsfromtheloft.com/comedy/trevor-no...,"Filmed at The Lincoln Theatre in Washington, D..."
15,https://scrapsfromtheloft.com/comedy/dave-atte...,[Cheers and applause] – you guys ready to meet...
16,https://scrapsfromtheloft.com/comedy/nate-barg...,Nate Bargatze: Hello World (2023)\nGenre: Come...
17,https://scrapsfromtheloft.com/comedy/neal-bren...,[Hip-hop music plays] [cheers and applause] ma...
18,https://scrapsfromtheloft.com/comedy/ellen-deg...,"Ladies and gentlemen, please, welcome Ellen De..."
19,https://scrapsfromtheloft.com/comedy/hannah-ei...,(vinyl record popping)\n\n(“J’ai Du L’Oublier”...


Hitting rate limits. Could slow down dramatically + add backoff, orrr just revert to the existing huggingface dataset. Was going to do this myself for fun but if it's going to be a pain maybe not worth it, huggingface version has >80% of them.

In [80]:
ds = load_dataset("zachgitt/comedy-transcripts")

INFO:httpx:HTTP Request: HEAD https://huggingface.co/datasets/zachgitt/comedy-transcripts/resolve/main/README.md "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/api/resolve-cache/datasets/zachgitt/comedy-transcripts/433e9c34e80a1f2f7f9fb30e7815bf147005ce0f/README.md "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/datasets/zachgitt/comedy-transcripts/resolve/433e9c34e80a1f2f7f9fb30e7815bf147005ce0f/comedy-transcripts.py "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: HEAD https://s3.amazonaws.com/datasets.huggingface.co/datasets/datasets/zachgitt/comedy-transcripts/zachgitt/comedy-transcripts.py "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/datasets/zachgitt/comedy-transcripts/resolve/433e9c34e80a1f2f7f9fb30e7815bf147005ce0f/.huggingface.yaml "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET https://datasets-server.huggingface.co/info?dataset=zachgitt/comedy-transcripts "HTTP/1.1 200 OK"


In [90]:
df = ds['train'].to_pandas().rename(columns=lambda x: x.replace('-', '_'))

In [98]:
df.transcript.sample().values[0]

"[Crowd chatter] Ladies and gentlemen: Bill Burr! [Cheers and applause] All right. Thank you! Thank you. Thank you, sir. How are ya? How’s it going? All right. [Cheers and applause] All right, all right, all right, all right. All right, everybody settle down. I wanna get a gun. I do. I really do. I never had that feeling before till I moved out to Los Angeles. This city just messes with your mind, you know? It’s overpopulated, technically doesn’t have a water supply. Right? The dollar’s crashing. Shit keeps you up at night. You’re just thinking… “What am I gonna do when the zombies come?” Right? Start reading up on shit. “Get some powdered food. Plant some zucchini. Get a windmill.” Right? And that’s all well and good, but if you don’t know how to fight, all you’re doing is gathering supplies for the toughest guy on the block. Right? I was thinking about that. What am I gonna do if some dude turns me upside down, starts shaking the gold coins outta my pockets? I gotta get a gun. So I’m

## Scratch

In [128]:
from aeon.secrets import SecretManager
from openai import OpenAI
from pydantic import BaseModel, Field
import re

In [108]:
class Response(BaseModel):
    
    joke: str = Field(
        ...,
        description="One or more sentences containing a joke from the "
                    "input text."
    )
        
    subtext: str = Field(
        ...,
        description="The often banal observation underlying the joke. "
                    "This should not be funny."
    )
        
    joke_variant: str = Field(
        ...,
        description="Your attempt to modify the original joke while "
                    "maintaining the same subtext."
    )

In [109]:
instructions = """
You are a detail-oriented research assistant with a shrewd understanding of humor, human behavior, and writing.
Your job is to extract jokes from the user's message.

  <instructions>You are one step in a data pipeline for a humor-related research project. Your task is to extract all jokes from the input passage and return valid JSON where each object contains "joke" and "subtext" fields. You can think of the subtext as the often banal point that the joke is making, which the comedian has ultimately massaged or restructured such that the resulting joke subverts the audience's expectations in a fun way. The subtext should not be funny, e.g. it might plainly state "airplane food is bad". Not every sentence in the input must belong to a joke, but most of them probably will because I am showing you standup transcripts.
      
  <example_input>
  [comedian: Louis CK]
  I could never wear white pants because I’ll get my period, first of all. I know that. [Laughter] Or diarrhea, more likely. [Laughter] Which is — That’s really my period. Diarrhea. About once a month, I’m like, “Oh, fuck, here we go.” [Laughter] “Better just get home. And don’t make any big decisions today.” [Laughter] It’s true. Don’t make — You know, if you have diarrhea, don’t, like, negotiate. It’s a bad bargaining position. If I have diarrhea, you stand between me and the toilet, I’ll sell you my house for 10 cents.
  </example_input>
  <example_output>
  [
      {
          "joke": "I could never wear white pants because I’ll get my period, first of all. I know that. Or diarrhea, more likely.",
          "subtext": "Wearing white pants makes any bodily mishap immediately obvious, so it’s risky."
      },
      {
          "joke": "Which is — That’s really my period. Diarrhea. About once a month, I’m like, “Oh, fuck, here we go.”",
          "subtext": "Monthly digestive mishaps can feel as routine and unavoidable as menstruation."
      },
      {
          "joke": "“Better just get home. And don’t make any big decisions today.” It’s true. Don’t make — You know, if you have diarrhea, don’t, like, negotiate. It’s a bad bargaining position. If I have diarrhea, you stand between me and the toilet, I’ll sell you my house for 10 cents.",
          "subtext": "Being desperate for a bathroom gives you zero leverage in any negotiation."
      }
  ]
  </example_output>
  </instructions>
"""

In [120]:
unlabeled = '.'.join(df.transcript.iloc[418].split('.')[:25]) + '.'
user_message = f"<example_to_label>{unlabeled}</example_to_label>"

In [124]:
secrets = SecretManager().get_secrets()

In [126]:
openai_client = OpenAI(api_key=secrets['OPENAI_API_KEY'])

In [130]:
OPENAI_KEY_PATTERN = r'sk-[a-zA-Z0-9_-]*[a-zA-Z0-9]{48}'

In [127]:
res = openai_client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=[
        {"role": "developer", "content": instructions},
        {"role": "user", "content": user_message}
    ],
    response_format=Response,
    temperature=0.3,
    logprobs=True
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 401 Unauthorized"


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-llm-l*****************************************************PrZb. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}